In [12]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "notebook-intro",
   "metadata": {},
   "source": [
    "# Sign Language Digits Recognition Analysis\n",
    "\n",
    "This notebook provides an interactive analysis of the sign language digits recognition project.\n",
    "\n",
    "## Setup Instructions:\n",
    "1. Ensure the dataset is placed in `data/raw/Dataset/` with folders 0/, 1/, 2/, ..., 9/\n",
    "2. Run cells sequentially\n",
    "3. Results will be saved in the `results/` directory"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "imports-setup",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import all necessary libraries\n",
    "import sys\n",
    "import os\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import warnings\n",
    "\n",
    "# Suppress warnings for cleaner output\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Add project directories to path\n",
    "sys.path.append('../src')\n",
    "sys.path.append('..')\n",
    "\n",
    "# Check if we can import the config\n",
    "try:\n",
    "    from config import *\n",
    "    print(\"✓ Config imported successfully\")\n",
    "except ImportError as e:\n",
    "    print(f\"❌ Config import failed: {e}\")\n",
    "    print(\"Creating basic config...\")\n",
    "    \n",
    "    # Basic config if import fails\n",
    "    BASE_DIR = os.path.dirname(os.path.abspath('..'))\n",
    "    DATA_DIR = os.path.join(BASE_DIR, 'data')\n",
    "    RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')\n",
    "    PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')\n",
    "    RESULTS_DIR = os.path.join(BASE_DIR, 'results')\n",
    "    PLOTS_DIR = os.path.join(RESULTS_DIR, 'plots')\n",
    "    \n",
    "    NUM_CLASSES = 10\n",
    "    IMAGE_SIZE = (64, 64)\n",
    "    \n",
    "    # Create directories\n",
    "    for dir_path in [DATA_DIR, RAW_DATA_DIR, PROCESSED_DATA_DIR, RESULTS_DIR, PLOTS_DIR]:\n",
    "        os.makedirs(dir_path, exist_ok=True)\n",
    "\n",
    "# Import project modules\n",
    "try:\n",
    "    from src.data_preprocessing import DataPreprocessor\n",
    "    from src.model_training import ModelTrainer\n",
    "    from src.evaluation import ModelEvaluator\n",
    "    from src.hyperparameter_tuning import HyperparameterTuner\n",
    "    from src.visualization import Visualizer\n",
    "    print(\"✓ All project modules imported successfully\")\n",
    "except ImportError as e:\n",
    "    print(f\"❌ Module import failed: {e}\")\n",
    "    print(\"Please ensure you're running this notebook from the notebooks/ directory\")\n",
    "    print(\"and that all source files are in the src/ directory\")\n",
    "\n",
    "# Set plotting style\n",
    "plt.style.use('default')\n",
    "plt.rcParams['figure.figsize'] = (12, 8)\n",
    "plt.rcParams['font.size'] = 10\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"NOTEBOOK SETUP COMPLETED\")\n",
    "print(\"=\"*50)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "data-exploration-header",
   "metadata": {},
   "source": [
    "## 1. Data Exploration and Preprocessing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "data-loading",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize preprocessor and visualizer\n",
    "print(\"Initializing components...\")\n",
    "preprocessor = DataPreprocessor()\n",
    "visualizer = Visualizer()\n",
    "\n",
    "print(\"Components initialized successfully!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "dataset-inspection",
   "metadata": {},
   "outputs": [],
   "source": [
    "# First, let's inspect the dataset structure\n",
    "print(\"Inspecting dataset structure...\")\n",
    "preprocessor.inspect_dataset_structure()\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"DATASET STRUCTURE INSPECTION COMPLETED\")\n",
    "print(\"=\"*50)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "dataset-loading",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the dataset using the updated method\n",
    "print(\"Loading Sign Language Digits dataset...\")\n",
    "print(\"-\" * 40)\n",
    "\n",
    "# Try to load the dataset\n",
    "success = preprocessor.load_kaggle_dataset()\n",
    "\n",
    "if success:\n",
    "    print(\"\\n✓ Dataset loaded successfully!\")\n",
    "    print(f\"Total samples: {len(preprocessor.image_data)}\")\n",
    "    print(f\"Image shape: {preprocessor.image_data.shape}\")\n",
    "    print(f\"Labels shape: {preprocessor.labels.shape}\")\n",
    "    print(f\"Image data type: {preprocessor.image_data.dtype}\")\n",
    "    print(f\"Labels data type: {preprocessor.labels.dtype}\")\n",
    "    print(f\"Image value range: [{preprocessor.image_data.min()}, {preprocessor.image_data.max()}]\")\n",
    "    print(f\"Label range: [{preprocessor.labels.min()}, {preprocessor.labels.max()}]\")\n",
    "    \n",
    "    # Verify data alignment\n",
    "    print(\"\\nVerifying data alignment...\")\n",
    "    alignment_ok = preprocessor.verify_data_alignment()\n",
    "    \n",
    "    if alignment_ok:\n",
    "        print(\"✓ Data alignment verification passed!\")\n",
    "    else:\n",
    "        print(\"❌ Data alignment issues detected!\")\n",
    "else:\n",
    "    print(\"\\n❌ Failed to load dataset!\")\n",
    "    print(\"\\nTroubleshooting steps:\")\n",
    "    print(\"1. Download dataset from: https://github.com/ardamavi/Sign-Language-Digits-Dataset\")\n",
    "    print(\"2. Extract and place 'Dataset' folder in data/raw/ directory\")\n",
    "    print(\"3. Ensure folder structure: data/raw/Dataset/0/, data/raw/Dataset/1/, ..., data/raw/Dataset/9/\")\n",
    "    print(\"4. Each digit folder should contain image files\")\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"DATASET LOADING COMPLETED\")\n",
    "print(\"=\"*50)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "sample-visualization",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize sample images from each class\n",
    "if preprocessor.image_data is not None:\n",
    "    print(\"Visualizing sample images...\")\n",
    "    \n",
    "    try:\n",
    "        visualizer.plot_sample_images(\n",
    "            preprocessor.image_data,\n",
    "            preprocessor.labels,\n",
    "            save_path=os.path.join(PLOTS_DIR, 'sample_images.png')\n",
    "        )\n",
    "        print(\"✓ Sample images visualization completed\")\n",
    "    except Exception as e:\n",
    "        print(f\"❌ Sample visualization failed: {e}\")\n",
    "        \n",
    "        # Alternative visualization\n",
    "        print(\"Trying alternative visualization...\")\n",
    "        preprocessor.visualize_samples(save_path=os.path.join(PLOTS_DIR, 'sample_images_alt.png'))\n",
    "else:\n",
    "    print(\"❌ Cannot visualize - no data loaded\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "class-distribution",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze class distribution\n",
    "if preprocessor.labels is not None:\n",
    "    print(\"Analyzing class distribution...\")\n",
    "    \n",
    "    try:\n",
    "        distribution = visualizer.plot_data_distribution(\n",
    "            preprocessor.labels,\n",
    "            save_path=os.path.join(PLOTS_DIR, 'class_distribution.png')\n",
    "        )\n",
    "        \n",
    "        print(\"\\nClass distribution:\")\n",
    "        for digit, count in distribution.items():\n",
    "            percentage = (count / len(preprocessor.labels)) * 100\n",
    "            print(f\"  Digit {digit}: {count} samples ({percentage:.1f}%)\")\n",
    "            \n",
    "        print(\"✓ Class distribution analysis completed\")\n",
    "    except Exception as e:\n",
    "        print(f\"❌ Distribution analysis failed: {e}\")\n",
    "        \n",
    "        # Alternative distribution analysis\n",
    "        print(\"Using alternative method...\")\n",
    "        distribution = preprocessor.get_class_distribution()\n",
    "else:\n",
    "    print(\"❌ Cannot analyze distribution - no labels loaded\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "data-preprocessing",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Normalize and split the data\n",
    "if preprocessor.image_data is not None:\n",
    "    print(\"Preprocessing data...\")\n",
    "    print(\"-\" * 30)\n",
    "    \n",
    "    try:\n",
    "        # Normalize data to [0, 1] range\n",
    "        print(\"1. Normalizing data...\")\n",
    "        preprocessor.normalize_data('standard')\n",
    "        print(f\"   After normalization: [{preprocessor.image_data.min():.3f}, {preprocessor.image_data.max():.3f}]\")\n",
    "        \n",
    "        # Split data into train/val/test\n",
    "        print(\"2. Splitting data...\")\n",
    "        preprocessor.split_data(test_size=0.15, val_size=0.15, random_state=42)\n",
    "        \n",
    "        print(f\"   Training samples: {preprocessor.X_train.shape[0]}\")\n",
    "        print(f\"   Validation samples: {preprocessor.X_val.shape[0]}\")\n",
    "        print(f\"   Test samples: {preprocessor.X_test.shape[0]}\")\n",
    "        \n",
    "        # Save processed data\n",
    "        print(\"3. Saving processed data...\")\n",
    "        preprocessor.save_processed_data()\n",
    "        \n",
    "        print(\"✓ Data preprocessing completed successfully!\")\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"❌ Data preprocessing failed: {e}\")\n",
    "        import traceback\n",
    "        traceback.print_exc()\n",
    "else:\n",
    "    print(\"❌ Cannot preprocess - no data loaded\")\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"DATA PREPROCESSING COMPLETED\")\n",
    "print(\"=\"*50)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "model-training-header",
   "metadata": {},
   "source": [
    "## 2. Model Training and Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "trainer-initialization",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize trainer and evaluator\n",
    "print(\"Initializing model trainer and evaluator...\")\n",
    "\n",
    "try:\n",
    "    trainer = ModelTrainer()\n",
    "    evaluator = ModelEvaluator()\n",
    "    \n",
    "    # Load preprocessed data\n",
    "    print(\"Loading preprocessed data...\")\n",
    "    data_loaded = trainer.load_data()\n",
    "    \n",
    "    if data_loaded:\n",
    "        print(\"✓ Preprocessed data loaded successfully\")\n",
    "        print(f\"Training data shape: {trainer.preprocessor.X_train.shape}\")\n",
    "        print(f\"Validation data shape: {trainer.preprocessor.X_val.shape}\")\n",
    "        print(f\"Test data shape: {trainer.preprocessor.X_test.shape}\")\n",
    "    else:\n",
    "        print(\"❌ Failed to load preprocessed data\")\n",
    "        print(\"Using data from current preprocessing...\")\n",
    "        trainer.preprocessor = preprocessor\n",
    "        \n",
    "    print(\"✓ Trainer and evaluator initialized\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ Initialization failed: {e}\")\n",
    "    import traceback\n",
    "    traceback.print_exc()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "model-training",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Train a basic CNN model\n",
    "print(\"Training CNN model...\")\n",
    "print(\"-\" * 30)\n",
    "\n",
    "try:\n",
    "    # Train model with data augmentation\n",
    "    print(\"Starting training...\")\n",
    "    history = trainer.train_model(\n",
    "        model_type='basic', \n",
    "        epochs=20, \n",
    "        batch_size=32,\n",
    "        use_augmentation=True,\n",
    "        verbose=1\n",
    "    )\n",
    "    \n",
    "    print(\"✓ Model training completed!\")\n",
    "    \n",
    "    # Display training summary\n",
    "    if history and hasattr(history, 'history'):\n",
    "        final_train_acc = history.history['accuracy'][-1]\n",
    "        final_val_acc = history.history['val_accuracy'][-1]\n",
    "        print(f\"Final training accuracy: {final_train_acc:.4f}\")\n",
    "        print(f\"Final validation accuracy: {final_val_acc:.4f}\")\n",
    "        \n",
    "except Exception as e:\n",
    "    print(f\"❌ Training failed: {e}\")\n",
    "    import traceback\n",
    "    traceback.print_exc()\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"MODEL TRAINING COMPLETED\")\n",
    "print(\"=\"*50)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "training-visualization",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot training history\n",
    "if 'history' in locals() and history is not None:\n",
    "    print(\"Visualizing training history...\")\n",
    "    \n",
    "    try:\n",
    "        trainer.plot_training_history(\n",
    "            save_path=os.path.join(PLOTS_DIR, 'training_history.png')\n",
    "        )\n",
    "        \n",
    "        # Also save training history data\n",
    "        trainer.save_training_history()\n",
    "        \n",
    "        print(\"✓ Training history visualization completed\")\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"❌ History visualization failed: {e}\")\n",
    "        \n",
    "        # Alternative visualization\n",
    "        print(\"Creating alternative training plots...\")\n",
    "        \n",
    "        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))\n",
    "        \n",
    "        # Plot accuracy\n",
    "        ax1.plot(history.history['accuracy'], label='Training Accuracy')\n",
    "        ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')\n",
    "        ax1.set_title('Model Accuracy')\n",
    "        ax1.set_xlabel('Epoch')\n",
    "        ax1.set_ylabel('Accuracy')\n",
    "        ax1.legend()\n",
    "        ax1.grid(True)\n",
    "        \n",
    "        # Plot loss\n",
    "        ax2.plot(history.history['loss'], label='Training Loss')\n",
    "        ax2.plot(history.history['val_loss'], label='Validation Loss')\n",
    "        ax2.set_title('Model Loss')\n",
    "        ax2.set_xlabel('Epoch')\n",
    "        ax2.set_ylabel('Loss')\n",
    "        ax2.legend()\n",
    "        ax2.grid(True)\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.savefig(os.path.join(PLOTS_DIR, 'training_history_alt.png'), dpi=300, bbox_inches='tight')\n",
    "        plt.show()\n",
    "else:\n",
    "    print(\"❌ No training history to visualize\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "model-evaluation",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate the trained model\n",
    "print(\"Evaluating model performance...\")\n",
    "print(\"-\" * 35)\n",
    "\n",
    "try:\n",
    "    # Get evaluation results\n",
    "    eval_results = trainer.evaluate_model()\n",
    "    \n",
    "    if eval_results:\n",
    "        # Calculate comprehensive metrics\n",
    "        metrics = evaluator.calculate_metrics(\n",
    "            eval_results['y_true_classes'],\n",
    "            eval_results['y_pred_classes'],\n",
    "            eval_results['y_pred']\n",
    "        )\n",
    "        \n",
    "        # Display key metrics\n",
    "        print(\"\\n📊 Model Performance Metrics:\")\n",
    "        print(f\"Test Accuracy:  {metrics['accuracy']:.4f} ({metrics['accuracy']*100:.2f}%)\")\n",
    "        print(f\"Precision (macro): {metrics['precision_macro']:.4f}\")\n",
    "        print(f\"Recall (macro):    {metrics['recall_macro']:.4f}\")\n",
    "        print(f\"F1-Score (macro):  {metrics['f1_macro']:.4f}\")\n",
    "        \n",
    "        print(\"\\n✓ Model evaluation completed successfully!\")\n",
    "        \n",
    "    else:\n",
    "        print(\"❌ Model evaluation failed - no results returned\")\n",
    "        \n",
    "except Exception as e:\n",
    "    print(f\"❌ Evaluation failed: {e}\")\n",
    "    import traceback\n",
    "    traceback.print_exc()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "confusion-matrix",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate confusion matrix\n",
    "if 'eval_results' in locals() and eval_results is not None:\n",
    "    print(\"Generating confusion matrix...\")\n",
    "    \n",
    "    try:\n",
    "        evaluator.plot_confusion_matrix(\n",
    "            eval_results['y_true_classes'],\n",
    "            eval_results['y_pred_classes'],\n",
    "            save_path=os.path.join(PLOTS_DIR, 'confusion_matrix.png')\n",
    "        )\n",
    "        print(\"✓ Confusion matrix generated\")\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"❌ Confusion matrix generation failed: {e}\")\n",
    "        \n",
    "        # Alternative confusion matrix\n",
    "        from sklearn.metrics import confusion_matrix\n",
    "        \n",
    "        cm = confusion_matrix(eval_results['y_true_classes'], eval_results['y_pred_classes'])\n",
    "        \n",
    "        plt.figure(figsize=(10, 8))\n",
    "        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', \n",
    "                   xticklabels=range(10), yticklabels=range(10))\n",
    "        plt.title('Confusion Matrix')\n",
    "        plt.xlabel('Predicted Label')\n",
    "        plt.ylabel('True Label')\n",
    "        plt.savefig(os.path.join(PLOTS_DIR, 'confusion_matrix_alt.png'), dpi=300, bbox_inches='tight')\n",
    "        plt.show()\n",
    "else:\n",
    "    print(\"❌ Cannot generate confusion matrix - no evaluation results\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "roc-curves",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate ROC curves\n",
    "if 'eval_results' in locals() and eval_results is not None:\n",
    "    print(\"Generating ROC curves...\")\n",
    "    \n",
    "    try:\n",
    "        evaluator.plot_roc_curves(\n",
    "            eval_results['y_true_classes'],\n",
    "            eval_results['y_pred'],\n",
    "            save_path=os.path.join(PLOTS_DIR, 'roc_curves.png')\n",
    "        )\n",
    "        print(\"✓ ROC curves generated\")\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"❌ ROC curves generation failed: {e}\")\n",
    "        import traceback\n",
    "        traceback.print_exc()\n",
    "else:\n",
    "    print(\"❌ Cannot generate ROC curves - no evaluation results\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "precision-recall-curves",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate Precision-Recall curves\n",
    "if 'eval_results' in locals() and eval_results is not None:\n",
    "    print(\"Generating Precision-Recall curves...\")\n",
    "    \n",
    "    try:\n",
    "        evaluator.plot_precision_recall_curves(\n",
    "            eval_results['y_true_classes'],\n",
    "            eval_results['y_pred'],\n",
    "            save_path=os.path.join(PLOTS_DIR, 'precision_recall_curves.png')\n",
    "        )\n",
    "        print(\"✓ Precision-Recall curves generated\")\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"❌ Precision-Recall curves generation failed: {e}\")\n",
    "        import traceback\n",
    "        traceback.print_exc()\n",
    "else:\n",
    "    print(\"❌ Cannot generate Precision-Recall curves - no evaluation results\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "hyperparameter-tuning-header",
   "metadata": {},
   "source": [
    "## 3. Hyperparameter Tuning Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "hyperparameter-tuner-init",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize hyperparameter tuner\n",
    "print(\"Initializing hyperparameter tuner...\")\n",
    "\n",
    "try:\n",
    "    tuner = HyperparameterTuner()\n",
    "    \n",
    "    # Load data for tuning\n",
    "    if not tuner.trainer.load_data():\n",
    "        print(\"Using current preprocessor data...\")\n",
    "        tuner.trainer.preprocessor = preprocessor\n",
    "    \n",
    "    print(\"✓ Hyperparameter tuner initialized\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ Tuner initialization failed: {e}\")\n",
    "    import traceback\n",
    "    traceback.print_exc()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "id": "batch-size-analysis",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze batch size effect (with reduced epochs for notebook)\n",
    "if 'tuner' in locals():\n",
    "    print(\"Analyzing batch size effect...\")\n",
    "    print(\"(Using reduced epochs for notebook demonstration)\")\n",
    "    print(\"-\" * 45)\n",
    "    \n",
    "    try:\n",
    "        # Temporarily modify epochs for faster notebook execution\n",
    "        original_epochs = tuner.trainer.epochs if hasattr(tuner.trainer, 'epochs') else 10\n",
    "        \n",
    "        batch_results = tuner.analyze_batch_size_effect(\n",
    "            batch_sizes=[16, 32, 64], \n",
    "            epochs=5  # Reduced for notebook\n",
    "        )\n",
    "        \n",
    "        if batch_results:\n",
    "            # Convert to DataFrame for better visualization\n",
    "            batch_df = pd.DataFrame(batch_results)\n",
    "            print(\"\\nBatch Size Analysis Results:\")\n",
    "            print(batch_df[['batch_size', 'final_val_accuracy', 'training_time']].round(4))\n",
    "            \n",
    "            # Visualize results\n",
    "            try:\n",
    "                visualizer.plot_hyperparameter_analysis(\n",
    "                    batch_results, \n",
    "                    'batch_size',\n",
    "                    save_path=os.path.join(PLOTS_DIR, 'batch_size_analysis.png')\n",
    "                )\n",
    "                print(\"✓ Batch size analysis visualization completed\")\n",
    "            except Exception as viz_e:\n",
    "                print(f\"Visualization failed: {viz_e}\")\n",
    "                \n",
    "        else:\n",
    "            print(\"❌ Batch size analysis returned no results\")\n",
    "            \n",
    "    except Exception as e:\n",
    "        print(f\"❌ Batch size analysis failed: {e}\")\n",
    "        import traceback\n",
    "        traceback.print_exc()\n",
    "else:\n",
    "    print(\"❌ Cannot run batch size analysis - tuner not initialized\")"

   ]
  }]}

{'cells': [{'cell_type': 'markdown',
   'id': 'notebook-intro',
   'metadata': {},
   'source': ['# Sign Language Digits Recognition Analysis\n',
    '\n',
    'This notebook provides an interactive analysis of the sign language digits recognition project.\n',
    '\n',
    '## Setup Instructions:\n',
    '1. Ensure the dataset is placed in `data/raw/Dataset/` with folders 0/, 1/, 2/, ..., 9/\n',
    '2. Run cells sequentially\n',
    '3. Results will be saved in the `results/` directory']},
  {'cell_type': 'code',
   'execution_count': 0,
   'id': 'imports-setup',
   'metadata': {},
   'outputs': [],
   'source': ['# Import all necessary libraries\n',
    'import sys\n',
    'import os\n',
    'import numpy as np\n',
    'import pandas as pd\n',
    'import matplotlib.pyplot as plt\n',
    'import seaborn as sns\n',
    'import warnings\n',
    '\n',
    '# Suppress warnings for cleaner output\n',
    "warnings.filterwarnings('ignore')\n",
    '\n',
    '# Add project directories to p